In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests as r
import matplotlib.pyplot as plt
import heapq
import queue
%matplotlib inline

In [ ]:
url = 'https://gist.githubusercontent.com/ctindall/4588786/raw/d3c331af8a4af0b3a36b3d27afc99042f1733038/us-constitution.txt'
text = r.get(url).text.strip().replace('\n', '')
textDict = dict(Counter(text))

df = pd.DataFrame({'char': list(textDict.keys()), 'char_count': list(textDict.values())})
df['freq'] = df['char_count'] * (1/df.char_count.sum())
df = df.sort_values(by='freq', ascending=True).reset_index(drop=True)
df['rank'] = df.index + 1

freq = [(row['char'], row['freq']) for index, row in df.iterrows()]
#freq = [('A', 0.5), ('B', 0.3), ('C', 0.1), ('D', 0.1)]

In [ ]:
df['initial_entropy'] = -1 * df['freq'] * np.log2(df['freq'])
print('Initial Self Entropy: {:.3f} bits'.format(sum(df['initial_entropy'])))

In [ ]:
def sorted_key(x):
    return x[1]

def huffman(ls):
    sorted_ls = sorted(ls, key = sorted_key)
    l2 = sorted_ls[:2]
    ls = sorted_ls[2:] + [([l2[0][0], l2[1][0]], l2[0][1] + l2[1][1])]
    if len(ls) == 1:
        return ls[0][0]
    else:
        return huffman(ls)

In [ ]:
tr = huffman(freq)

In [ ]:
def find_char(ls, char):
    if ls[0] == char:
        return '0'
    elif ls[1] == char:
        return '1'
    elif type(ls[0]) == list:
        attempt = find_char(ls[0], char)
        if attempt != None:
            return '0' + attempt
        else:
            return None
    elif type(ls[1]) == list:
        if find_char(ls[1], char) != None:
            return '1' + find_char(ls[1], char)
        else:
            return None
    else:
        return None

In [ ]:
code = {char: find_char(tr, char) for char in text}
code

In [ ]:
#code = {'A': '0', 'B': '11', 'C': '100', 'D': '101'}

In [ ]:
text = 'ABCDBCBCBCAABr'
encoded = ''
for i in range(len(text)):
    encoded += code[text[i]]
print(encoded)

In [ ]:
#plt.figure(figsize=(20, 10))
#plt.bar(df.char, df.freq, color='blue', alpha=0.5)
#plt.yticks(fontsize=14)
#plt.xticks(fontsize=14)
#plt.xlabel('Character', fontsize=16)
#plt.ylabel('Character Frequency', fontsize=16)
#plt.title('Character Frequency Chart of the US Constitution', fontsize=19)
#plt.show()